In [7]:
import numpy as np
import pprint
from gridworld import GridworldEnv

In [8]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

def policy_eval(policy, env: GridworldEnv , discount_factor = 1.0 , theta = 1e-6):
    """
    Evaluate a policy given an environment and a full description
    """
    # start with random value function
    values = np.zeros(env.number_of_states)
    while True:
        delta = 0
        for state in range(env.number_of_states):
            value = 0
            for action, action_prob in enumerate(policy[state]):
                for prob, next_state, reward, done in env.P[state][action]:
                    value += action_prob * prob * (reward + discount_factor * values[next_state])
            delta = max(delta, np.abs(value - values[state]))
            values[state] = value
        if delta < theta:
            break
    return np.array(values)
        
    

In [12]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    def one_step_lookahead(state, V):
        A = np.zeros(env.nA)
        for action in range(env.nA):
            for prob, next_state, reward, done in env.P[state][action]:
                A[action] += prob * (reward + discount_factor * V[next_state])
        return A
    
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        V = policy_eval_fn(policy, env, discount_factor) # evaluate current policy
        policy_stable = True
        for s in range(env.nS):
            old_action = np.argmax(policy[s])
            action_values = one_step_lookahead(s, V)
            new_action = np.argmax(action_values)
            if old_action != new_action:
                policy_stable = False
            policy[s] = np.eye(env.nA)[new_action]
        if policy_stable:
            return policy, V

In [13]:
policy, V = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(V)
print("")

print("Reshaped Grid Value Function:")
print(V.reshape(env.shape))
print("")

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [14]:
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
print(np.testing.assert_array_almost_equal(V, expected_v, decimal=2))

None


In [9]:
expected_v

array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])

In [15]:
V.astype(int).flatten()
V[3] = 31

In [16]:
print(np.testing.assert_array_almost_equal(V, expected_v, decimal=2))

AssertionError: 
Arrays are not almost equal to 2 decimals

Mismatched elements: 1 / 16 (6.25%)
Max absolute difference: 34.
Max relative difference: 11.33333333
 x: array([ 0., -1., -2., 31., -1., -2., -3., -2., -2., -3., -2., -1., -3.,
       -2., -1.,  0.])
 y: array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])